# Checkpoint 7: N64 Emulator Setup for RunPod

This notebook guides you through setting up the Mupen64Plus emulator on a RunPod GPU instance for Mario Kart 64 reinforcement learning.

**Important Notes:**
- This setup is designed for **RunPod Linux x86_64** instances with NVIDIA GPUs
- You must provide your own **legally obtained** Mario Kart 64 ROM file
- The ROM file is not included and cannot be distributed due to copyright restrictions
- Ensure you own a physical copy of Mario Kart 64 before proceeding

**Learning Objectives:**
1. Set up a virtual display for headless rendering
2. Install and configure Mupen64Plus emulator
3. Understand the gym wrapper architecture
4. Verify emulator performance on GPU instances

---
## 1. System Information

First, let's verify our system environment and GPU availability.

In [ ]:
import platform
import sys
import os

print("=" * 50)
print("SYSTEM INFORMATION")
print("=" * 50)
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.machine()}")
print(f"Python Version: {sys.version}")
print(f"Working Directory: {os.getcwd()}")
print("\n" + "=" * 50)
print("GPU INFORMATION")
print("=" * 50)

In [ ]:
!nvidia-smi

---
## 2. Install System Dependencies

We need several system packages for the emulator and virtual display.

In [ ]:
%%bash
# Update package list and install system dependencies
apt-get update -qq

# Install required packages:
# - xvfb: Virtual framebuffer for headless display
# - libsdl2-dev: SDL2 library for graphics/input
# - git: Version control for cloning repos
# - cmake: Build system for compiling from source
# - build-essential: GCC, make, and other build tools
apt-get install -y -qq \
    xvfb \
    libsdl2-dev \
    git \
    cmake \
    build-essential \
    libgl1-mesa-glx \
    libglu1-mesa \
    libpng-dev \
    libfreetype6-dev

echo "System dependencies installed successfully!"

---
## 3. Start Virtual Framebuffer (Xvfb)

Since RunPod instances typically run headless (no physical display), we need a virtual framebuffer to render graphics.

In [ ]:
%%bash
# Kill any existing Xvfb process
pkill Xvfb 2>/dev/null || true

# Start Xvfb on display :99
# -screen 0: Define screen 0
# 1024x768x24: Resolution 1024x768 with 24-bit color depth
Xvfb :99 -screen 0 1024x768x24 &

# Wait for Xvfb to start
sleep 2

# Verify Xvfb is running
if pgrep -x "Xvfb" > /dev/null; then
    echo "Xvfb started successfully on display :99"
else
    echo "ERROR: Xvfb failed to start"
    exit 1
fi

In [ ]:
# Set the DISPLAY environment variable for this Python session
import os
os.environ['DISPLAY'] = ':99'
print(f"DISPLAY environment variable set to: {os.environ.get('DISPLAY')}")

---
## 4. Clone gym-mupen64plus Repository

This repository provides a Gymnasium-compatible wrapper for the Mupen64Plus emulator.

In [ ]:
%%bash
# Create workspace directory if it doesn't exist
mkdir -p /workspace

# Remove existing gym-mupen64plus directory if present
rm -rf /workspace/gym-mupen64plus

# Clone the gym-mupen64plus repository
cd /workspace
git clone https://github.com/bzier/gym-mupen64plus.git

# Verify clone was successful
if [ -d "/workspace/gym-mupen64plus" ]; then
    echo "Repository cloned successfully!"
    ls -la /workspace/gym-mupen64plus/
else
    echo "ERROR: Failed to clone repository"
fi

---
## 5. Install Mupen64Plus Emulator

Install the Mupen64Plus N64 emulator from Ubuntu repositories.

In [ ]:
%%bash
# Install mupen64plus and its plugins
apt-get install -y -qq \
    mupen64plus \
    mupen64plus-audio-sdl \
    mupen64plus-input-sdl \
    mupen64plus-rsp-hle \
    mupen64plus-video-rice \
    mupen64plus-video-glide64mk2

echo "Mupen64Plus installation complete!"

# Show installed version
mupen64plus --version 2>&1 | head -5 || echo "Version check requires ROM file"

---
## 6. Install Python Dependencies

Install the Python packages needed for the RL environment.

In [ ]:
!pip install -q gymnasium numpy opencv-python pygame

In [ ]:
# Verify installations
import gymnasium
import numpy as np
import cv2
import pygame

print(f"gymnasium version: {gymnasium.__version__}")
print(f"numpy version: {np.__version__}")
print(f"opencv version: {cv2.__version__}")
print(f"pygame version: {pygame.__version__}")

---
## 7. ROM Setup Instructions

**IMPORTANT: You must provide your own legally obtained ROM file.**

### Option 1: Upload via JupyterLab
1. In the JupyterLab file browser (left panel), navigate to `/workspace/roms/`
2. Click the upload button (up arrow icon)
3. Select your `MarioKart64.z64` or `MarioKart64.n64` ROM file

### Option 2: Using wget (if ROM is hosted)
```bash
# If you have the ROM hosted somewhere (e.g., your own server):
mkdir -p /workspace/roms
wget -O /workspace/roms/MarioKart64.z64 YOUR_ROM_URL
```

### Option 3: Using SCP/SFTP
```bash
# From your local machine:
scp /path/to/MarioKart64.z64 root@YOUR_POD_IP:/workspace/roms/
```

### Expected ROM Details
- **Filename**: `MarioKart64.z64` or `MarioKart64.n64`
- **Expected Size**: ~12 MB (12,582,912 bytes for .z64)
- **Region**: US (NTSC) recommended for consistent RAM addresses

In [ ]:
%%bash
# Create the roms directory
mkdir -p /workspace/roms
echo "ROM directory created at /workspace/roms/"
echo "Please upload your ROM file to this directory."

---
## 8. Verify ROM Installation

In [ ]:
import os
import hashlib

ROM_DIR = "/workspace/roms"
EXPECTED_EXTENSIONS = ['.z64', '.n64', '.v64']

def find_rom_files(directory):
    """Search for N64 ROM files in the specified directory."""
    rom_files = []
    
    if not os.path.exists(directory):
        print(f"Directory {directory} does not exist!")
        return rom_files
    
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            ext = os.path.splitext(filename)[1].lower()
            if ext in EXPECTED_EXTENSIONS:
                size_mb = os.path.getsize(filepath) / (1024 * 1024)
                rom_files.append((filename, size_mb))
    
    return rom_files

# Check for ROM files
print("Checking for ROM files...")
print("=" * 50)

roms = find_rom_files(ROM_DIR)

if roms:
    print("ROM files found:")
    for rom_name, rom_size in roms:
        print(f"  - {rom_name} ({rom_size:.2f} MB)")
        
        # Check if it looks like Mario Kart 64
        if 'mario' in rom_name.lower() and 'kart' in rom_name.lower():
            print(f"    -> This appears to be Mario Kart 64!")
            if 11 < rom_size < 14:
                print(f"    -> Size looks correct for Mario Kart 64")
            else:
                print(f"    -> WARNING: Unexpected size (expected ~12 MB)")
else:
    print("No ROM files found!")
    print(f"\nPlease upload your ROM to: {ROM_DIR}")
    print("Supported formats: .z64, .n64, .v64")

---
## 9. Test Mupen64Plus Installation

In [ ]:
%%bash
# Test that mupen64plus is installed and accessible
echo "Testing mupen64plus installation..."
echo "=" | head -c 50 && echo

# Show help information
mupen64plus --help 2>&1 | head -30

echo ""
echo "=" | head -c 50 && echo
echo "Mupen64plus is installed and ready!"

---
## 10. Wrapper Architecture Explanation

### How gym-mupen64plus Works

The gym wrapper provides a standard Gymnasium interface to the N64 emulator:

```
+-------------------+     +------------------+     +----------------+
|   Your RL Agent   | --> | gym-mupen64plus  | --> |  Mupen64Plus   |
|  (Python code)    |     |    (Wrapper)     |     |   (Emulator)   |
+-------------------+     +------------------+     +----------------+
         ^                        |                       |
         |                        v                       v
         |                 +-----------+           +-------------+
         +-----------------|Observation|<----------|  N64 Game   |
           action          | (pixels/  |           | (Mario Kart)|
                           |   RAM)    |           +-------------+
                           +-----------+
```

### Key Components

1. **Gymnasium Environment**: Standard `env.step()`, `env.reset()` interface
2. **Input Handler**: Converts discrete/continuous actions to N64 controller inputs
3. **Screen Capture**: Grabs frames from the emulator for visual observations
4. **RAM Reader**: Reads game memory for reward calculation and state info
5. **Reward Shaper**: Calculates rewards based on game state (speed, position, etc.)

### Action Space
The N64 controller has:
- Analog stick (X, Y axes)
- A, B buttons
- Z trigger, L/R shoulders
- C buttons (up, down, left, right)
- D-pad
- Start button

For Mario Kart, we typically use:
- Analog stick for steering
- A button for acceleration
- B button for brake
- Z/R for items/hop

---
## 11. FPS Performance Test

This test measures how fast the emulator can run on your GPU instance. Higher FPS means faster training!

In [ ]:
# FPS Performance Test Placeholder
# This test requires a valid ROM file to run

import time

def fps_test_placeholder():
    """
    Placeholder for FPS performance testing.
    
    When the emulator environment is set up with a valid ROM,
    this function will:
    1. Create the environment
    2. Run random actions for N steps
    3. Measure and report FPS
    """
    print("FPS Performance Test")
    print("=" * 50)
    print("")
    print("This test requires:")
    print("1. A valid Mario Kart 64 ROM in /workspace/roms/")
    print("2. The gym-mupen64plus environment to be configured")
    print("")
    print("Once your ROM is ready, uncomment and run the test code below.")
    print("")
    print("Expected performance on RunPod GPUs:")
    print("- RTX 3090: ~200-400 FPS")
    print("- RTX 4090: ~300-500 FPS")
    print("- A100: ~400-600 FPS")
    
fps_test_placeholder()

In [ ]:
# Simulated FPS test (demonstrates what the real test would look like)

import time
import numpy as np

def simulated_fps_test(n_steps=1000):
    """
    Simulates an FPS test to demonstrate the testing methodology.
    
    In a real environment, this would interact with the emulator.
    """
    print(f"Running simulated FPS test ({n_steps} steps)...")
    
    # Simulate environment step times
    # Real emulator would take ~2-5ms per step on good hardware
    simulated_step_time = 0.003  # 3ms per step = ~333 FPS
    
    start_time = time.time()
    
    for i in range(n_steps):
        # Simulate action selection
        action = np.random.randint(0, 5)
        
        # Simulate environment step
        time.sleep(simulated_step_time * np.random.uniform(0.8, 1.2))
        
        # Simulate observation (would be screen capture)
        obs = np.random.randint(0, 255, (64, 64, 3), dtype=np.uint8)
        
        if (i + 1) % 200 == 0:
            elapsed = time.time() - start_time
            current_fps = (i + 1) / elapsed
            print(f"  Step {i+1}: {current_fps:.1f} FPS")
    
    total_time = time.time() - start_time
    avg_fps = n_steps / total_time
    
    print("\n" + "=" * 50)
    print(f"Simulated Test Complete!")
    print(f"Total steps: {n_steps}")
    print(f"Total time: {total_time:.2f}s")
    print(f"Average FPS: {avg_fps:.1f}")
    print("=" * 50)
    print("\nNote: This is a SIMULATION. Real FPS depends on your GPU.")

# Run the simulated test
simulated_fps_test(n_steps=500)

---
## 12. Environment Test Code

**Uncomment the code below when your ROM is ready.**

In [ ]:
# ============================================================
# ENVIRONMENT TEST CODE
# Uncomment this code when your ROM is ready
# ============================================================

# import gymnasium as gym
# import numpy as np
# import sys
# sys.path.append('/workspace/gym-mupen64plus')

# # Configuration
# ROM_PATH = "/workspace/roms/MarioKart64.z64"  # Update with your ROM path

# def test_environment():
#     """
#     Test the Mario Kart 64 environment.
#     """
#     print("Creating Mario Kart 64 environment...")
#     
#     # Create environment (adjust based on gym-mupen64plus API)
#     # env = gym.make('MarioKart64-v0', rom_path=ROM_PATH)
#     
#     # Reset environment
#     # obs, info = env.reset()
#     # print(f"Observation shape: {obs.shape}")
#     # print(f"Action space: {env.action_space}")
#     
#     # Run a few random steps
#     # for i in range(100):
#     #     action = env.action_space.sample()
#     #     obs, reward, terminated, truncated, info = env.step(action)
#     #     
#     #     if (i + 1) % 20 == 0:
#     #         print(f"Step {i+1}: reward={reward:.2f}")
#     #     
#     #     if terminated or truncated:
#     #         break
#     
#     # env.close()
#     # print("Environment test complete!")

# # Run the test
# test_environment()

print("Environment test code is commented out.")
print("Uncomment the code above when your ROM is ready.")

In [ ]:
# ============================================================
# REAL FPS PERFORMANCE TEST
# Uncomment this code when your ROM is ready
# ============================================================

# import time
# import gymnasium as gym
# import numpy as np
# import sys
# sys.path.append('/workspace/gym-mupen64plus')

# def real_fps_test(n_steps=1000):
#     """
#     Actual FPS performance test with the emulator.
#     """
#     ROM_PATH = "/workspace/roms/MarioKart64.z64"
#     
#     print(f"Starting FPS test ({n_steps} steps)...")
#     
#     # Create environment
#     # env = gym.make('MarioKart64-v0', rom_path=ROM_PATH)
#     # obs, info = env.reset()
#     
#     # start_time = time.time()
#     # 
#     # for i in range(n_steps):
#     #     action = env.action_space.sample()
#     #     obs, reward, terminated, truncated, info = env.step(action)
#     #     
#     #     if terminated or truncated:
#     #         obs, info = env.reset()
#     #     
#     #     if (i + 1) % 200 == 0:
#     #         elapsed = time.time() - start_time
#     #         fps = (i + 1) / elapsed
#     #         print(f"Step {i+1}: {fps:.1f} FPS")
#     # 
#     # total_time = time.time() - start_time
#     # avg_fps = n_steps / total_time
#     # 
#     # env.close()
#     # 
#     # print(f"\nFinal Results:")
#     # print(f"Average FPS: {avg_fps:.1f}")
#     # print(f"Step time: {1000/avg_fps:.2f}ms")

# # real_fps_test(1000)

print("Real FPS test code is commented out.")
print("Uncomment the code above when your ROM is ready.")

---
## 13. Troubleshooting Guide

### Common Issues and Solutions

#### Display Errors

**Problem**: `Error: Unable to open display` or `cannot open display`

**Solutions**:
1. Ensure Xvfb is running:
   ```bash
   pgrep -x Xvfb || (Xvfb :99 -screen 0 1024x768x24 &)
   ```

2. Set DISPLAY variable:
   ```python
   import os
   os.environ['DISPLAY'] = ':99'
   ```

3. Check if port 99 is in use:
   ```bash
   # Try a different display number
   Xvfb :98 -screen 0 1024x768x24 &
   export DISPLAY=:98
   ```

---

#### ROM Not Found Errors

**Problem**: `ROM file not found` or `Invalid ROM`

**Solutions**:
1. Verify ROM location:
   ```bash
   ls -la /workspace/roms/
   ```

2. Check file permissions:
   ```bash
   chmod 644 /workspace/roms/*.z64
   ```

3. Verify ROM is not corrupted:
   ```bash
   # Check file size (should be ~12MB for Mario Kart 64)
   ls -lh /workspace/roms/MarioKart64.z64
   ```

4. Try different ROM format:
   - `.z64` (big-endian, most common)
   - `.n64` (little-endian)
   - `.v64` (byte-swapped)

---

#### Emulator Crashes

**Problem**: Emulator crashes during startup or gameplay

**Solutions**:
1. Try a different video plugin:
   ```bash
   mupen64plus --gfx mupen64plus-video-rice /path/to/rom.z64
   # or
   mupen64plus --gfx mupen64plus-video-glide64mk2 /path/to/rom.z64
   ```

2. Check GPU memory:
   ```bash
   nvidia-smi
   ```

3. Reduce resolution:
   ```bash
   # In mupen64plus config, set lower resolution
   # ~/.config/mupen64plus/mupen64plus.cfg
   ```

4. Run with verbose logging:
   ```bash
   mupen64plus --verbose /path/to/rom.z64
   ```

---

#### Performance Issues

**Problem**: Low FPS or stuttering

**Solutions**:
1. Use frame skip:
   ```python
   # Skip every N frames to speed up training
   env = gym.make('MarioKart64-v0', frame_skip=4)
   ```

2. Lower observation resolution:
   ```python
   # Resize observations to smaller size
   env = gym.wrappers.ResizeObservation(env, (64, 64))
   ```

3. Disable rendering when not needed:
   ```python
   # Only enable for debugging
   env = gym.make('MarioKart64-v0', render_mode=None)
   ```

---

#### Python Import Errors

**Problem**: `ModuleNotFoundError: No module named 'gym_mupen64plus'`

**Solutions**:
1. Add to Python path:
   ```python
   import sys
   sys.path.append('/workspace/gym-mupen64plus')
   ```

2. Install the package:
   ```bash
   cd /workspace/gym-mupen64plus
   pip install -e .
   ```

---
## 14. Knowledge Check Quiz

Test your understanding of the emulator setup!

In [ ]:
def emulator_setup_quiz():
    """
    Interactive quiz to test understanding of emulator setup.
    """
    questions = [
        {
            "question": "What is Xvfb used for in this setup?",
            "options": [
                "A) GPU acceleration",
                "B) Virtual framebuffer for headless display",
                "C) Network connectivity",
                "D) ROM validation"
            ],
            "correct": "B",
            "explanation": "Xvfb (X Virtual Framebuffer) provides a virtual display for rendering graphics on servers without physical monitors."
        },
        {
            "question": "What is the typical file extension for N64 ROM files?",
            "options": [
                "A) .rom",
                "B) .bin",
                "C) .z64, .n64, or .v64",
                "D) .iso"
            ],
            "correct": "C",
            "explanation": "N64 ROMs use .z64 (big-endian), .n64 (little-endian), or .v64 (byte-swapped) extensions."
        },
        {
            "question": "What does the DISPLAY=:99 environment variable do?",
            "options": [
                "A) Sets the screen resolution to 99 pixels",
                "B) Tells applications to use X display number 99",
                "C) Limits FPS to 99",
                "D) Sets memory buffer size"
            ],
            "correct": "B",
            "explanation": "DISPLAY=:99 tells applications to connect to X display number 99, which is where Xvfb is listening."
        },
        {
            "question": "Why do we use gym-mupen64plus instead of running the emulator directly?",
            "options": [
                "A) The emulator doesn't work without it",
                "B) It provides a standard Gymnasium interface for RL",
                "C) It's faster than the raw emulator",
                "D) It includes the ROM file"
            ],
            "correct": "B",
            "explanation": "gym-mupen64plus wraps the emulator with a standard Gymnasium API (step, reset, action_space, etc.) making it easy to use with RL algorithms."
        },
        {
            "question": "What is a typical expected FPS on a modern GPU for this emulator?",
            "options": [
                "A) 30-60 FPS",
                "B) 200-600 FPS",
                "C) 1000-2000 FPS",
                "D) 10-20 FPS"
            ],
            "correct": "B",
            "explanation": "Modern GPUs can run the N64 emulator at 200-600 FPS, which is much faster than the original 30 FPS, enabling faster RL training."
        }
    ]
    
    score = 0
    total = len(questions)
    
    print("=" * 60)
    print("EMULATOR SETUP QUIZ")
    print("=" * 60)
    print("\nAnswer each question by entering A, B, C, or D.\n")
    
    for i, q in enumerate(questions, 1):
        print(f"\nQuestion {i}/{total}: {q['question']}")
        for option in q['options']:
            print(f"  {option}")
        
        while True:
            try:
                answer = input("\nYour answer: ").strip().upper()
                if answer in ['A', 'B', 'C', 'D']:
                    break
                print("Please enter A, B, C, or D")
            except:
                print("Input error. Using placeholder answer.")
                answer = q['correct']  # For non-interactive mode
                break
        
        if answer == q['correct']:
            print("Correct!")
            score += 1
        else:
            print(f"Incorrect. The correct answer is {q['correct']}.")
        
        print(f"Explanation: {q['explanation']}")
    
    print("\n" + "=" * 60)
    print(f"QUIZ COMPLETE!")
    print(f"Your score: {score}/{total} ({100*score/total:.0f}%)")
    print("=" * 60)
    
    if score == total:
        print("\nPerfect score! You're ready to start training!")
    elif score >= total * 0.7:
        print("\nGood job! Review the sections you missed before proceeding.")
    else:
        print("\nConsider reviewing this notebook again before continuing.")

# Run the quiz
print("To take the quiz, uncomment and run: emulator_setup_quiz()")
# emulator_setup_quiz()

---
## Summary

In this notebook, we:

1. **Checked system information** and verified GPU availability
2. **Installed system dependencies** (Xvfb, SDL2, build tools)
3. **Started a virtual framebuffer** for headless rendering
4. **Cloned gym-mupen64plus** for the Gymnasium wrapper
5. **Installed Mupen64Plus** emulator and plugins
6. **Set up Python dependencies** (gymnasium, numpy, opencv, pygame)
7. **Learned about ROM setup** and verification
8. **Understood the wrapper architecture** and how it interfaces with RL agents
9. **Prepared FPS performance testing** code
10. **Reviewed troubleshooting** for common issues

### Next Steps

1. Upload your legally obtained Mario Kart 64 ROM to `/workspace/roms/`
2. Uncomment and run the environment test code
3. Run the FPS performance test to establish baseline
4. Proceed to the next notebook for RAM reading and game state extraction

---

*Remember: You must own a legal copy of Mario Kart 64 to use the ROM file.*